# Advent of Code 2022 - Day 15

In [ ]:
import re

In [ ]:
inp = open('input.txt','r').read() 

In [ ]:
d = [[int(y) for y in re.findall('-?\d+\.?\d*',x)] for x in inp.splitlines()]
# [sensor x, sensor y , closest x, closest y]

In [ ]:
class sensor:
    """
    
    """

    def __init__(self, x, y, closest_x, closest_y):
        self.x = x
        self.y = y
        
        #beacon position
        self.b_x = closest_x
        self.b_y = closest_y
        
        self.mhtn = abs(self.x-self.b_x) + abs(self.y - self.b_y)
        
    def mhtn_dist(self,point):
        px,py = point
        return abs(self.x - px) + abs(self.y - py)
    
    #calculate if a point is within or equal to manhattan distance of a point
    def blocked_point(self, point):
        if self.mhtn_dist(point) <= self.mhtn:
            return True
        else:
            return False

In [ ]:
sensors = []
for x,y,b_x,b_y in d:
    sensors.append(sensor(x,y,b_x,b_y))

In [ ]:
min_x = sensors[0].x
max_x = sensors[0].x

In [ ]:
for s in sensors:
    if (s.x - s.mhtn) < min_x:
        min_x = s.x - s.mhtn
        
    if (s.x + s.mhtn)  > max_x:
        max_x = s.x + s.mhtn

In [ ]:
results = []
check_y = 10
loop = True

for i in range(min_x,max_x):
    check_point = (i,check_y)
    d = 0
    for s in sensors:
        if (s.b_x,s.b_y) == check_point:
            break
        if s.blocked_point(check_point) == True:
            #print(check_point," blocked by ",(s.x,s.y))
            results.append([i,1])
            d = 1
            break
    if d==0:
        results.append([i,0])

In [ ]:
sum([b for a,b in results])

### Part 2

My brute-force approach has no chance for part 2 where we would have 4,000,000 by 4,000,000 cells to check.

Need to iterate over the sensors and work out the 'ruled-out' segments.

In [ ]:
check_y = 2000000

In [ ]:
# take a list of sensors, and return the range of x values that are 'blocked' for a fixed y value.
# also return a set of known locations (these are where our beacons are located)

def blocked_ranges(sensors,check_y):
    ranges = []
    known = set()
    for s in sensors:

        x,y = s.x,s.y
        m = s.mhtn

        #reaches check_y 
        r = m - abs(check_y-y) 

        if r < 0:
            continue

        min_x = x - r
        max_x = x + r

        ranges.append((min_x,max_x))

        if s.b_y == check_y :
            known.add(s.b_x)
            
        ranges.sort()
    return ranges, known

In [ ]:
# now we consolidate our list of sorted exclusion ranges

def consol_ranges(ranges):
    final = []

    for left,right in ranges:

        #if first pass
        if not final:
            final.append([left,right])
            continue
        
        #want to compare new range to previous one
        prev_left,prev_right = final[-1]
        
        # if new left hand number is bigger than the end of the previous right +1 , we start a new range as we do not have a continous range 
        if left > prev_right +1:
            final.append([left,right])
            continue
        
        # If we have a continous range, update the right hand number to encompass this new range
        final[-1][1] = max(right,prev_right)
    
    return final

In [ ]:
# see if it works for part 1 - e.g. over 1 Y value
def part_1(final,known):
    
    blocked = set()
    for left, right in final:
        for x in range(left, right + 1):
            blocked.add(x)

    return (len(blocked - known))

In [ ]:
# Part 1 answer
part_1(final,known)

In [ ]:
# Part 2
#` Still a big loop but o(M) rather than o(M^2)

In [ ]:
size = 4000000

In [ ]:
def part_2(sensors,size):

    for check_y in range(size + 1):

        ranges, known = blocked_ranges(sensors,check_y)

        final = consol_ranges(ranges)

        # for reach range in final, check whether there is a value between 0 and size which is not inside a range
        a = 0 
        for left, right in final:
            if a < left:
                return (a * size + check_y)


            #update value to one past the current range
            a = max(a, right + 1)
            if a > size:
                break 

In [ ]:
part_2(sensors,size)